In [11]:
from auxiliary_functions import *
from bert_model import *
import pickle
from transformers import BertTokenizer, BertConfig
from transformers import DataCollatorWithPadding

device = torch.device("cuda:0")
model_name = "google-bert/bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
model = pickle.load(open("model.pickle", "rb"))

In [32]:
df = pd.read_csv("data/funda_data_11_06_2024.csv")
df = df[["descrip", "price"]]
dataset = create_dataset(df, 10000, tokenizer, 3)
dataset = dataset.remove_columns(["__index_level_0__"])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/9522 [00:00<?, ? examples/s]

{'train': (7617, 5), 'test': (1905, 5)}


In [33]:
from tqdm import tqdm

def test_pred(x):
    input_x = {key: torch.as_tensor([val], device=device) for key, val in x.items()}
    outputs = model(**input_x)

    return outputs["logits"].detach().cpu()

vals = []
for d in tqdm(dataset["test"]):
    vals += test_pred(d)

100%|██████████| 1905/1905 [01:16<00:00, 24.89it/s]


In [39]:
torch.as_tensor(vals)

tensor([-0.0961, -0.3007,  2.0938,  ...,  1.0427, -0.1708, -0.3301])

In [44]:
print("RMSE=",rmse_loss(torch.as_tensor(vals), torch.as_tensor(dataset["test"]["price"])).item())
print("MSE=",mse_loss(torch.as_tensor(vals), torch.as_tensor(dataset["test"]["price"])).item())
print("MAE=",l1_loss(torch.as_tensor(vals), torch.as_tensor(dataset["test"]["price"])).item())

RMSE= 0.6819419264793396
MSE= 0.4650447964668274
MAE= 0.45871007442474365
